In [3]:
import sys
import polars as pl
import plotly.graph_objects as go
sys.path.append("..")

from src.queries import query_demand_evolution, query_workshops_locations, query_stores_locations, query_sales_stocks
from src.dataloaders import get_postgres_uri

In [2]:
def plot_map(df_store: pl.DataFrame, df_workshop: pl.DataFrame):
    """
    Plot a visually stunning map with stores and workshops locations.
    """
    fig = go.Figure()

    # --- STORES TRACE ---
    fig.add_trace(go.Scattermap(
        name="Stores",
        mode="markers+text",
        lat=df_store.get_column("s_latitude").to_list(),
        lon=df_store.get_column("s_longitude").to_list(),
        text=df_store.get_column("s_name").to_list(),
        marker=dict(
            symbol=['commercial'] * df_store.height,
            size=16,
            color='rgb(91, 192, 222)', # A calm, professional blue
        ),
        textfont=dict(
            # family='Arial, sans-serif',
            size=10,
            color='rgb(0, 0, 0)'
        ),
        textposition="top center",
        hovertemplate='<b>Store</b>: %{text}<extra></extra>'
    ))

    # --- WORKSHOPS TRACE ---
    fig.add_trace(go.Scattermap(
        name="Workshops",
        mode="markers+text",
        lat=df_workshop.get_column("w_latitude"),
        lon=df_workshop.get_column("w_longitude"),
        text=df_workshop.get_column("w_name"),
        customdata=df_workshop.get_column("w_capacity"),
        marker=dict(
            symbol=['industry'] * df_workshop.height,
            size=18,
            color='rgb(240, 173, 78)', # A warm, industrial orange
        ),
        textfont=dict(
            # family='Arial, sans-serif',
            size=10,
            color='rgb(0, 0, 0)'
        ),
        textposition="top center",
        hovertemplate='<b>Workshop</b>: %{text}<br><b>Capacity</b>: %{customdata} units<extra></extra>'
    ))

    # --- LAYOUT ENHANCEMENTS ---
    fig.update_layout(
        # title=dict(
        #     text='<b>Operational Footprint: Stores & Workshops</b>',
        #     y=0.95,
        #     x=0.5,
        #     xanchor='center',
        #     yanchor='top',
        #     font=dict(size=20, family='Arial, sans-serif', color='rgb(60,60,60)')
        # ),
        margin=dict(l=10, r=10, t=10, b=10),
        height=600,
        width=1000,
        autosize=True,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.01,
            xanchor="center",
            x=0.5,
            font=dict(family='Arial, sans-serif', size=12)
        ),
        map=dict(
            style="carto-positron", # A clean, modern map style
            center=dict(lat=-0.210799, lon=-78.481357),
            pitch=0,
            bearing=-80, # Standard north-up orientation
            zoom=11
        ),
        paper_bgcolor='rgb(255, 255, 255)',
        plot_bgcolor='rgb(255, 255, 255)'
    )

    fig.show()

In [3]:

df_stores = pl.read_database_uri(query=query_stores_locations, uri=get_postgres_uri())
df_workshops = pl.read_database_uri(query=query_workshops_locations, uri=get_postgres_uri())
plot_map(df_stores, df_workshops)

In [ ]:
def plot_inventory_balance(df_demand_evol: pl.DataFrame, df_sales_stocks: pl.DataFrame):
    """Plot the inventory balance for each product in each store over time.
    The balance is calculated as the difference between expected initial inventory, expected deliveries,
    expected demand fulfilled, and expected demand unfulfilled.

    Args:
        df_demand_evol (pl.DataFrame): DataFrame containing the demand evolution data with the following columns:
            - c_date: date of the demand evolution
            - p_name: name of the product
            - s_name: name of the store
            - init_stocks: expected initial inventory
            - units_sent: expected deliveries
            - met_demand: expected demand fulfilled
            - unmet_demand: expected demand unfulfilled
    """

    products_stores = df_demand_evol.unique(subset=["p_name", "s_name"]).select("p_name", "s_name").sort(["s_name","p_name"]).rows()

    for product, store in products_stores:
        df_subset = df_demand_evol.filter(p_name=product, s_name=store)
        df_subset_ss = df_sales_stocks.filter(p_name=product, s_name=store)

        fig = go.Figure()

        # --- DATA TRACES with new color palette ---

        # Inventory (Historical)
        fig.add_trace(go.Bar(
            x=df_subset_ss["c_date"], y=df_subset_ss["units"],
            name='Inventory', marker_color='rgb(252, 186, 3)', opacity=1,
            hovertemplate='<b>Date</b>: %{x}<br><b>Inventory</b>: %{y} units<extra></extra>'
        ))

        # Initial Inventory (Projected)
        fig.add_trace(go.Bar(
            x=df_subset["c_date"], y=df_subset["initial_stocks"],
            name='Projected Initial Inventory', marker_color='rgb(252, 186, 3)', opacity=0.6,
            hovertemplate='<b>Date</b>: %{x}<br><b>Projected Initial Inv.</b>: %{y} units<extra></extra>'
        ))

        # Deliveries (Projected)
        fig.add_trace(go.Bar(
            x=df_subset["c_date"], y=df_subset["expected_units"],
            name='Projected Deliveries', marker_color='rgb(3, 148, 252)', opacity=0.7,
            hovertemplate='<b>Date</b>: %{x}<br><b>Projected Deliveries</b>: %{y} units<extra></extra>'
            # name='Projected Deliveries', marker_color='rgb(52, 73, 94)', opacity=0.7,
            # hovertemplate='<b>Date</b>: %{x}<br><b>Projected Deliveries</b>: %{y} units<extra></extra>'
        ))

        # Sales (Historical)
        fig.add_trace(go.Bar(
            x=df_subset_ss["c_date"], y=-df_subset_ss["units_sold"],
            name='Sales', marker_color='rgb(23, 191, 99)', opacity=1,
            hovertemplate='<b>Date</b>: %{x}<br><b>Sales</b>: %{customdata} units<extra></extra>',
            customdata=df_subset_ss["units_sold"]
        ))

        # Met Demand (Projected)
        fig.add_trace(go.Bar(
            x=df_subset["c_date"], y=-df_subset["met_demand"],
            name='Projected Fulfilled Sales', marker_color='rgb(23, 191, 99)', opacity=0.6,
            hovertemplate='<b>Date</b>: %{x}<br><b>Projected Fulfilled</b>: %{customdata} units<extra></extra>',
            customdata=df_subset["met_demand"]
        ))

        # Unmet Demand (Projected)
        fig.add_trace(go.Bar(
            x=df_subset["c_date"], y=-df_subset["unmet_demand"],
            name='Projected Unfulfilled Sales', marker_color='rgb(52, 73, 94)', opacity=0.6,
            hovertemplate='<b>Date</b>: %{x}<br><b>Projected Unfulfilled</b>: %{customdata} units<extra></extra>',
            customdata=df_subset["unmet_demand"]
        ))

        # --- LAYOUT ENHANCEMENTS ---
        fig.update_traces(marker_line_width=0)

        fig.update_layout(
            # title=f'<b>Inventory & Sales Projection: {product} at {store}</b>',
            barmode='relative',
            paper_bgcolor='rgb(248, 248, 255)',
            plot_bgcolor='rgb(248, 248, 255)',
            xaxis=dict(
                title='Date',
                showgrid=False,
                tickfont=dict(family='Arial, sans-serif', size=12, color='rgb(100,100,100)')
            ),
            yaxis=dict(
                title='Units',
                showgrid=True,
                gridcolor='rgb(220,220,220)',
                tickfont=dict(family='Arial, sans-serif', size=12, color='rgb(100,100,100)')
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                font=dict(family='Arial, sans-serif', size=10)
            ),
            font=dict(family='Arial, sans-serif', color='rgb(60,60,60)'),
            margin=dict(l=60, r=30, t=80, b=60)
        )
        # save to HTML file to export it later to streamlit
        fig.write_html(f"inventory_balance_{product}_{store}.html")
        fig.show()

In [10]:
query = query_demand_evolution.format(date_from='2016-08-15')
df_demand_evol = pl.read_database_uri(query=query, uri=get_postgres_uri())
df_sales = pl.read_database_uri(query=query_sales_stocks.format(date_from='2016-08-15'), uri=get_postgres_uri())
plot_inventory_balance(df_demand_evol, df_sales)

In [ ]:
total_sales = 100
unmet_demand = 20

labels = ['Fulfilled Sales', 'Unfulfilled Sales']
values = [total_sales, unmet_demand]
colors = ["rgb(23, 191, 99)", "rgb(52, 73, 94)"]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4, marker_colors=colors, opacity=0.6),])
fig.update_layout(
    height=300,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

NameError: name 'unmet_demand' is not defined